In [1]:
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn as nn
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,TensorDataset
from sklearn.metrics import accuracy_score

In [2]:
from itertools import product 

lr=[0.1,0.01,0.001,0.0001]
hidden_size=[32,64,128,512,1024]
embed_size=[16,32,64,128,256]
batch_size=[8,32,64,128,256]
parameters=pd.DataFrame(
    list(product(lr, hidden_size, embed_size,batch_size)),
    columns=['lr', 'hidden', 'embed','batch']
)
parameters.to_csv('parameters.csv')



In [3]:
tweets=pd.read_csv('Tweets.csv')
tweets.head(12)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


In [4]:
#tweets.info()

In [5]:
data=tweets['text'].values
labels=tweets['airline_sentiment'].values
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
label=le.fit_transform(labels)
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data,label,test_size=0.10,random_state=42)

In [6]:
data[11]

'@VirginAmerica I &lt;3 pretty graphics. so much better than minimal iconography. :D'

In [7]:
import re
def preprocessing(s):
    # Remove all non-word characters (everything except numbers and letters)
    #s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)
    
    return s

In [8]:
from nltk.corpus import stopwords

def tokenizer(x_train,x_test):
    word_list=[]
    stop_word=set(stopwords.words('english'))
    for sent in x_train:
        sent=sent.lower().split()[1:]
        for word in sent:
            word=preprocessing(word)
            if word not in stop_word and word != '' : 
                    word_list.append(word)
                
                
    corpus=Counter(word_list)     
    corpus_=sorted(corpus,key=corpus.get,reverse=True)
    one_hot_dict={w:i+1 for i,w in enumerate(corpus_)}
    
   # tockenize

    train_list=[]
    test_list=[]
#    for  i,sent in enumerate( x_train):
#         for word in sent.lower().split()[1:] :
#            word=preprocessing(word)
#            if word in one_hot_dict:
#                train_list.append(one_hot_dict[word])
        
    for  sent in  x_train:   
         train_list.append( [one_hot_dict[preprocessing(word)] for word in sent.lower().split()[1:]
                           if preprocessing(word) in one_hot_dict.keys()])
        
    for sent in x_test :
        test_list.append( [one_hot_dict[preprocessing(word)]  for word in sent.lower().split()[1:]
                            if preprocessing(word) in one_hot_dict.keys()])
    
    
    
    return  train_list,test_list,one_hot_dict
        


In [9]:
x_tr,x_ts,vocab=tokenizer(x_train,x_test)

In [10]:
a=[len(x) for x in x_tr]
max_len_index=a.index(max(a))
len(x_tr[max_len_index])

23

In [11]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)  #37500,500
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

X_train=padding_(x_tr,25)
X_test=padding_(x_ts,25)

In [12]:
X_train[0].shape

(25,)

In [13]:
X_trainn[0].shape

NameError: name 'X_trainn' is not defined

In [578]:
def padding(data,pad):
    for i,sent in enumerate(data):
        feature=np.zeros((1,pad),dtype=int)
        if len(np.array(sent)) != 0:
            feature[:,-len(sent):]=np.array(sent)
            data[i]=feature
        else :
            data[i]=feature
    return data

X_trainn=padding(x_tr,25)
X_testt=padding(x_ts,25)

In [395]:
#DataLoader?

In [586]:
class sentimentLSTM(nn.Module):
    def __init__(self,num_layers,batch_size,hidden_size,vocab_size,embed_size):
        super(sentimentLSTM,self).__init__()
        
        self.hidden=hidden_size
        self.embed=nn.Embedding(vocab_size,embed_size)
        self.embed_size=embed_size
        self.num_layers=num_layers
        
        self.lstm=nn.LSTM(input_size=self.embed_size,
                         hidden_size=self.hidden,
                         num_layers=self.num_layers,
                         batch_first=True)
        

        
        
        self.linear=nn.Linear(self.hidden,3)
        self.drop=nn.Dropout(0.3)
        self.soft=nn.Softmax()
        self.relu=nn.ReLU()
    

    
    def forward(self,x,hidden):
        
        batch=x.shape[0]
        x=x.view(batch,-1)
        x=self.embed(x)
        x,hidden=self.lstm(x)
        x=x.contiguous().view(-1, self.hidden)
        x=self.drop(x)
        x=self.linear(x)
        x=self.relu(x)
        #x=self.linear2(x)
        
        x=self.soft(x)
        out=x.view(batch,25, -1)   
        
        return out[:,-1] ,h

    def init_hidden(self,b):

        h0 = torch.zeros((self.num_layers,batch_size,self.hidden)).to(device)
        c0 = torch.zeros((self.num_layers,batch_size,self.hidden)).to(device)
        hidden = (h0,c0)
        return hidden

In [568]:
epochs=20
lr=0.01
hidden_size=1024
vocab_size=len(vocab)+1
embed_size=64
num_layers=2
model=sentimentLSTM(num_layers,batch_size,hidden_size,vocab_size,embed_size).to(device)
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


batch_size=16
train_data = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
test_data = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

train_loader=DataLoader(dataset=train_data,
                        shuffle=True,
                        batch_size=batch_size)
test_loader=DataLoader(dataset=test_data,
                        shuffle=True,
                        batch_size=batch_size)


for i in range(2):
    model.train()
    train_loss=0
    tr_loss_list=[]
    all_pred=[]
    h = model.init_hidden(batch_size)

    for inputs,label in train_loader:


        inputs=inputs.type(torch.LongTensor).to(device)
        label=label.type(torch.LongTensor).to(device)

        h = tuple([each.data for each in h])

        pred,h=model(inputs,h)
        all_pred.append(np.array([torch.argmax(i).item() for i in pred]))

        loss=criterion(pred,label)
        train_loss=+loss.item()
        
        model.zero_grad()
        loss.backward()
        optimizer.step()

    acc=accuracy_score(y_train,np.concatenate(all_pred))
    print('epochs {} accurary {:.3f}'.format(i,acc))

<ipython-input-567-e9d15fa7b0c7>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


KeyboardInterrupt: 

In [553]:
text=' I &lt;3 pretty graphics. so much better than minimal iconography'



In [555]:
word_seq = np.array([vocab[preprocessing(word)] for word in text.lower().split() 
                 if preprocessing(word)  in vocab.keys()])

feature=np.zeros((1,25),dtype=int)
feature[:,-len(word_seq):]=np.array(word_seq)

inputs =  torch.from_numpy(feature).type(torch.LongTensor).to(device)

model.eval()
with torch.no_grad():
    h = model.init_hidden(batch_size)
    h = tuple([each.data for each in h])
    output, h = model(inputs, h)

print(output)
le.inverse_transform([torch.argmax(output).item()])

tensor([[1.0000e+00, 6.8281e-08, 6.8281e-08]])


<ipython-input-541-e9d15fa7b0c7>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


array(['negative'], dtype=object)

In [514]:
parameters=pd.read_csv('parameters.csv').iloc[:,1:]

    
for  index,i in enumerate(np.array(parameters)):


    batch_size=int(i[3])
    train_data = TensorDataset(torch.Tensor(X_train), torch.Tensor(y_train))
    test_data = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))

    train_loader=DataLoader(dataset=train_data,
                        shuffle=True,
                        batch_size=batch_size)
    test_loader=DataLoader(dataset=test_data,
                        shuffle=True,
                        batch_size=batch_size)

    param_acc={}
    epochs=1
    lr=float(i[0])
    hidden_size=int(i[1])
    vocab_size=len(vocab)+1
    embed_size=int(i[2])
    num_layers=1
    model=sentimentLSTM(num_layers,batch_size,hidden_size,vocab_size,embed_size).to(device)
    criterion=nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)



    for i in range(epochs):
        model.train()
        train_loss=0
        tr_loss_list=[]
        all_pred=[]
        h = model.init_hidden(batch_size)

        for inputs,label in train_loader:

            model.zero_grad()

            inputs=inputs.type(torch.LongTensor).to(device)
            label=label.type(torch.LongTensor).to(device)

            h = tuple([each.data for each in h])

            pred,h=model(inputs,h)
            all_pred.append(np.array([torch.argmax(i).item() for i in pred]))

            loss=criterion(pred,label)
            train_loss=+loss.item()

            loss.backward()
            optimizer.step()

        acc=accuracy_score(y_train,np.concatenate(all_pred))
    param_acc.update({index:acc})
        #print('epochs {} accurary {:.3f}'.format(i,acc))
    print('index {} done acc {} '.format(index,acc))



<ipython-input-442-0f59259d06d4>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


index 0 done acc 0.5908469945355191 


<ipython-input-442-0f59259d06d4>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


index 1 done acc 0.6162720097146327 


<ipython-input-442-0f59259d06d4>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


...

index 3 done acc 0.6098967820279295 


<ipython-input-442-0f59259d06d4>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


index 4 done acc 0.6232544019429266 


<ipython-input-442-0f59259d06d4>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x=self.soft(x)


KeyboardInterrupt: 

In [ ]:
inputs

In [819]:
i=4
for inputs,label in train_loader:
    if i>1 :
        break

In [ ]:
def predict_text(text):
        word_seq = np.array([vocab[preprocess_string(word)] for word in text.split() 
                         if preprocessing(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)# reshape(1,-1)
        inputs=torch.from_numpy(padding_(word_seq,40)).to(device)

        output= model(inputs)
        return(output.item())